In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/Colab Notebooks/3yr_pjt

/content/drive/MyDrive/Colab Notebooks/3yr_pjt


Import modeules

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os

# Data pre-processing

## Stock price dataset preprocessing

In [ ]:
# Set the path for the original stock data and the directory for the preprocessed data
stock_dataset = './data/stock_data'
preprocessed_dataset = './data/stock_price_data'

# Create a directory for storing preprocessed data if it does not exist
if not os.path.exists(preprocessed_dataset):
    os.makedirs(preprocessed_dataset)

# Loop through each file in the original stock data directory
for file in os.listdir(stock_dataset):
    file_path = os.path.join(stock_dataset, file)
    # Process only CSV files
    if file_path.endswith('.csv'):
        # Read the CSV file, setting 'Date' as the index column and parsing dates
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Replace 0 values in 'Volume' and 'Close' columns with NaN and then drop rows with any NaN values
        df['Volume'] = df['Volume'].replace(0, np.nan)
        df['Close'] = df['Close'].replace(0, np.nan)
        df = df.dropna()

        # Define the path for saving the preprocessed file
        preprocessed_file_path = os.path.join(preprocessed_dataset, file)

        # Save the preprocessed data to a new CSV file
        df.to_csv(preprocessed_file_path)

## Feature dataset preprocessing (for Deep Learning)

Preprocess individual market index datasets (S&P 500, DJI, NASDAQ) in a similar manner

Replace 0 values, drop NaNs, remove commas, and convert to float

### S&P 500 dataset preprocessing

In [ ]:
snp500_dataset = 'data/snp500.csv'
df = pd.read_csv(snp500_dataset, index_col='Date', parse_dates=True)

df = df['Close*'].replace(0, np.nan)
df = df.dropna()
df = df.str.replace(',', '').astype(float)

df.to_csv('data/snp500_preprocessed.csv')

### Dow Jones Industrial Average (DJI) dataset preprocessing

In [ ]:
dji_dataset = 'data/DJI.csv'
df = pd.read_csv(dji_dataset, index_col='Date', parse_dates=True)

df = df['Close*'].replace(0, np.nan)
df = df.dropna()
df = df.str.replace(',', '').astype(float)

df.to_csv('data/dji_preprocessed.csv')

### Nasdaq (IXIC) dataset preprocessing

In [ ]:
nasdaq_dataset = 'data/IXIC.csv'
df = pd.read_csv(snp500_dataset, index_col='Date', parse_dates=True)

df = df['Close*'].replace(0, np.nan)
df = df.dropna()
df = df.str.replace(',', '').astype(float)

df.to_csv('data/ixic_preprocessed.csv')

## Combine feature datasets with stock dataset

In [ ]:
# Load the preprocessed market index datasets
snp500_preprocessed = pd.read_csv('data/snp500_preprocessed.csv', index_col='Date', parse_dates=True)
dji_preprocessed = pd.read_csv('data/dji_preprocessed.csv', index_col='Date', parse_dates=True)
nasdaq_preprocessed = pd.read_csv('data/ixic_preprocessed.csv', index_col='Date', parse_dates=True)

# Find the latest start date among the three datasets to align them temporally
latest_start_date = max(snp500_preprocessed.index.min(), dji_preprocessed.index.min(), nasdaq_preprocessed.index.min())

# Trim each dataset to start from this latest start date for consistency
snp500_preprocessed = snp500_preprocessed[snp500_preprocessed.index >= latest_start_date]
dji_preprocessed = dji_preprocessed[dji_preprocessed.index >= latest_start_date]
nasdaq_preprocessed = nasdaq_preprocessed[nasdaq_preprocessed.index >= latest_start_date]

# Rename columns to clearly indicate the source of the data
snp500_preprocessed.rename(columns={'Close*': 'Close_snp500'}, inplace=True)
dji_preprocessed.rename(columns={'Close*': 'Close_dji'}, inplace=True)
nasdaq_preprocessed.rename(columns={'Close*': 'Close_nasdaq'}, inplace=True)

preprocessed_dataset = './data/stock_price_data'

# Create a directory for the final merged datasets if it does not exist
deep_dataset = './data/deep_stock_data'
if not os.path.exists(deep_dataset):
    os.makedirs(deep_dataset)

# Merge each individual stock file with the preprocessed market index data
for file in os.listdir(preprocessed_dataset):
    file_path = os.path.join(preprocessed_dataset, file)
    # Process only CSV files
    if file.endswith('.csv'):
        stock_df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
        stock_df = stock_df[stock_df.index >= latest_start_date]

        # Join the stock data with each of the market index datasets
        merged_df = stock_df.join(snp500_preprocessed, how='left', rsuffix='_snp500')
        merged_df = merged_df.join(dji_preprocessed, how='left', rsuffix='_dji')
        merged_df = merged_df.join(nasdaq_preprocessed, how='left', rsuffix='_nasdaq')

        # Forward-fill any missing values in the market index columns
        for col in ['Close_snp500', 'Close_dji', 'Close_nasdaq']:
            merged_df[col] = merged_df[col].fillna(method='ffill')

        # Save the merged dataset to a new CSV file in the final directory
        merged_file_path = os.path.join(deep_dataset, file)
        merged_df.to_csv(merged_file_path)

### Normalization for deep learning

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Set the directory containing the deep preprocessed stock data
deep_dataset = './data/deep_stock_data'
# Initialize an empty DataFrame to combine all stock data
combined_df = pd.DataFrame()

# Combine data from all CSV files in the deep dataset directory into a single DataFrame
for file in os.listdir(deep_dataset):
    file_path = os.path.join(deep_dataset, file)
    # Ensure only CSV files are processed
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
        combined_df = pd.concat([combined_df, df])

# Select specific columns for scaling
combined_df = combined_df[['Volume', 'Close', 'Close_snp500', 'Close_dji', 'Close_nasdaq']]


### Save Scaler

In [ ]:
# Dictionary to store scalers for each column
scalers = {}
for col in combined_df.columns:
    # Initialize a MinMaxScaler for each column
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Fit the scaler to the column data
    scalers[col] = scaler.fit(combined_df[[col]])

    # Save each scaler to a file for later use
    joblib.dump(scaler, f'./scalers/{col}_scaler.pkl')


In [ ]:
# Directory for storing normalized datasets
normalized_dataset = './data/normalized_stock_data'
# Create the directory if it does not exist
if not os.path.exists(normalized_dataset):
    os.makedirs(normalized_dataset)

# Normalize and save each individual stock file using the fitted scalers
for file in os.listdir(deep_dataset):
    file_path = os.path.join(deep_dataset, file)
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Apply the saved scalers to the respective columns
        for col in scalers:
            scaler = scalers[col]
            df[col] = scaler.transform(df[[col]])

        # Define the path for the normalized file
        normalized_file_path = os.path.join(normalized_dataset, file)
        # Save the normalized data to a new CSV file
        df.to_csv(normalized_file_path)

# Train & Evaluate Model

## Error index

In [4]:
def get_mse(actual, forecast):
    actual = np.where(actual < 0, 0, actual)
    forecast = np.where(forecast < 0, 0, forecast)
    actual = actual + 0.0001
    forecast = forecast + 0.0001
    squared_diff = np.square(actual - forecast)
    return np.mean(squared_diff)

def get_mape(actual, forecast):
    actual = np.where(actual<0, 0, actual)
    forecast = np.where(forecast<0, 0, forecast)
    actual = actual + 0.0001
    forecast = forecast + 0.0001
    return np.mean(np.abs((actual - forecast) / actual)) * 100

def get_directional_accuracy(actual, forecast):
    # Calculate direction of actual price changes
    actual_direction = np.sign(np.diff(actual))
    # Calculate direction of forecasted price changes
    forecast_direction = np.sign(np.diff(forecast))

    correct_predictions = (actual_direction == forecast_direction).sum()
    total_predictions = len(actual_direction)
    directional_accuracy = correct_predictions / total_predictions
    return directional_accuracy

## Prophet

In [ ]:
from prophet import Prophet

In [ ]:
def evaluate_prophet():
    stock_price = './data/stock_price_data/'
    accuracy_df = pd.DataFrame(columns=['TICKER', 'mse', 'mape'])

    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)

        # Extract ticker from the file name
        ticker = os.path.splitext(file)[0]

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Extract 'Close' column and reset the index
        df_prophet = df['Close'].reset_index()

        # Rename the columns to match Prophet's requirements
        df_prophet.columns = ['ds', 'y']

        # Convert the 'ds' column to datetime
        df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

        # Split the data into train and evaluate sets
        cutoff_date = '2023-01-03'
        train = df_prophet[df_prophet['ds'] < cutoff_date]
        evaluate = df_prophet[df_prophet['ds'] >= cutoff_date]

        # Create and fit the Prophet model using the training data
        model = Prophet()
        model.fit(train)

        # Make future DataFrame for forecasting
        future = model.make_future_dataframe(periods=len(evaluate))

        # Forecast using the fitted model
        forecast = model.predict(future)

        # Extract actual and forecast values for evaluation
        y_true = evaluate['y'].values
        fcst = forecast.iloc[-len(evaluate):,:][['ds', 'yhat']]

        # Save the prediction data to a CSV file
        prediction_file_name = f"./prediction/{ticker}_prophet.csv"
        fcst.to_csv(prediction_file_name, index=False)

        # Accuracy
        y_pred = fcst['yhat'].values
        mse = get_mse(y_true, y_pred)
        mape = get_mape(y_true, y_pred)
        directional_accuracy = get_directional_accuracy(y_true, y_pred)

        # Store accuracy values in the DataFrame
        accuracy_df = pd.concat([accuracy_df, pd.DataFrame([[ticker, mse, mape, directional_accuracy]],
                                                    columns=['TICKER', 'mse', 'mape', 'directional_accuracy'])], ignore_index=True)

    return(accuracy_df)

In [ ]:
accuracy_prophet = evaluate_prophet()
accuracy_prophet.to_csv("prophet.csv")

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf5extm9v/lt6xs_uk.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf5extm9v/nep1km37.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=87382', 'data', 'file=/tmp/tmpf5extm9v/lt6xs_uk.json', 'init=/tmp/tmpf5extm9v/nep1km37.json', 'output', 'file=/tmp/tmpf5extm9v/prophet_modelvg8w73ev/prophet_model-20240219220012.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
22:00:12 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
22:00:19 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf5

## ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
def evaluate_arima():
    stock_price = './data/stock_price_data/'
    accuracy_df = pd.DataFrame(columns=['TICKER', 'mse', 'mape'])

    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)

        # Extract ticker from the file name
        ticker = os.path.splitext(file)[0]

        # Read the CSV file into a DataFrame
        df_arima = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Extract 'Close' column and reset the index
        df_arima = df_arima['Close'].reset_index()

        # Rename the columns for ARIMA
        df_arima.columns = ['ds', 'y']

        # Convert the 'ds' column to datetime
        df_arima['ds'] = pd.to_datetime(df_arima['ds'])

        # Split the data into train and evaluate sets
        cutoff_date = '2023-01-01'
        train = df_arima[df_arima['ds'] < cutoff_date]
        evaluate = df_arima[df_arima['ds'] >= cutoff_date]

        order = (1,1,0)

        # Fit ARIMA model using the training data with the optimal order
        model_arima = ARIMA(train['y'], order=order)
        results_arima = model_arima.fit()

        # Forecast using the fitted ARIMA model
        forecast = results_arima.get_forecast(steps=len(evaluate))
        y_pred = forecast.predicted_mean.values

        # Save the prediction data to a CSV file
        prediction_file_name = f"./prediction/{ticker}_arima.csv"
        pd.DataFrame({'ds': evaluate['ds'], 'yhat': y_pred}).to_csv(prediction_file_name, index=False)

        # Accuracy for ARIMA
        mse = get_mse(evaluate['y'].values, y_pred)
        mape = get_mape(evaluate['y'].values, y_pred)
        directional_accuracy = get_directional_accuracy(evaluate['y'].values, y_pred)

        # Update DataFrame with new metrics
        accuracy_df = pd.concat([accuracy_df, pd.DataFrame([[ticker, mse, mape, directional_accuracy]],
                                                    columns=['TICKER', 'mse', 'mape', 'directional_accuracy'])], ignore_index=True)

    return(accuracy_df)

In [ ]:
accuracy_arima = evaluate_arima()
accuracy_arima.to_csv("arima.csv")

## LSTM

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostRegressor
import joblib

In [6]:
# Function to create a sequence dataset for time series forecasting
def make_sequence_dataset(feature, label, window_size):
    feature_list = []
    label_list = []

    for i in range(len(feature)-window_size):

        feature_list.append(feature[i:i+window_size])
        label_list.append(label[i+window_size])

    return np.array(feature_list), np.array(label_list)

In [ ]:
# Function to build an LSTM model with specified input shape
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, activation='tanh', input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
def evaluate_lstm():
    # Set the path for the normalized stock data and model saving directory
    stock_price = './data/normalized_stock_data/'
    model_save_path = './models/'
    os.makedirs(model_save_path, exist_ok=True)

    # DataFrame to store accuracy metrics for each stock
    accuracy_df = pd.DataFrame(columns=['TICKER', 'mse', 'mape'])

    # Load the scaler used to normalize the 'Close' prices
    scaler_close = joblib.load('./scalers/Close_scaler.pkl')

    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)
        df = pd.read_csv(file_path, thousands=',')

        # Selecting feature and label columns
        feature_cols = ['Close', 'Volume', 'Close_snp500', 'Close_dji', 'Close_nasdaq']
        label_cols = ['Close']

        # Preparing the feature and label matrices
        feature_df = pd.DataFrame(df, columns=feature_cols)
        label_df = pd.DataFrame(df, columns=label_cols)

        # Converting DataFrame to numpy array for processing
        feature_np = feature_df.to_numpy()
        label_np = label_df.to_numpy()

        # Creating sequences from the data
        window_size = 40
        X, Y = make_sequence_dataset(feature_np, label_np, window_size)

        # Splitting the dataset into training and testing sets
        split = -218
        x_train = X[0:split]
        y_train = Y[0:split]
        x_test = X[split:]
        y_test = Y[split:]

        # Building and training the LSTM model
        model = build_lstm_model(x_train[0].shape)

        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

        # Training model with early stopping and learning rate reduction on plateau
        model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=32, callbacks=[early_stopping, reduce_lr])

        # Making predictions
        pred = model.predict(x_test)

        # De-normalizing the predictions and actual values for evaluation
        actual_denormalized = scaler_close.inverse_transform(y_test).flatten()
        pred_denormalized = scaler_close.inverse_transform(pred).flatten()

        # Extracting the ticker symbol from the file name for reference
        ticker = os.path.splitext(file)[0]

        # Saving the trained model
        model_save_filename = f"{model_save_path}{ticker}_lstm_model.h5"
        model.save(model_save_filename)

        # Calculate accuracy metrics (MSE, MAPE, and directional accuracy)
        mse = get_mse(actual_denormalized, pred_denormalized)
        mape = get_mape(actual_denormalized, pred_denormalized)
        directional_accuracy = get_directional_accuracy(actual_denormalized, pred_denormalized)

        # Save the forecast data to a CSV file
        forecast_file_name = f"./prediction/{ticker}_lstm.csv"
        pd.DataFrame({'ds': df['Date'].iloc[-len(pred_denormalized):], 'yhat': pred_denormalized}).to_csv(forecast_file_name, index=False)

        # Store accuracy values in the DataFrame
        accuracy_df = pd.concat([accuracy_df, pd.DataFrame([[ticker, mse, mape, directional_accuracy]], columns=['TICKER', 'mse', 'mape', 'directional_accuracy'])], ignore_index=True)

    return accuracy_df

In [ ]:
accuracy_lstm = evaluate_lstm()
accuracy_lstm.to_csv("lstm.csv")

Epoch 1/30
243/243 [==============================] - 9s 13ms/step - loss: 2.0052e-04 - mae: 0.0071 - val_loss: 5.7528e-06 - val_mae: 0.0021 - lr: 0.0010
Epoch 2/30
243/243 [==============================] - 2s 9ms/step - loss: 2.7547e-05 - mae: 0.0036 - val_loss: 3.0266e-05 - val_mae: 0.0050 - lr: 0.0010
Epoch 3/30
243/243 [==============================] - 2s 7ms/step - loss: 1.5464e-05 - mae: 0.0028 - val_loss: 1.0007e-05 - val_mae: 0.0026 - lr: 0.0010
Epoch 4/30
243/243 [==============================] - 2s 7ms/step - loss: 1.1226e-05 - mae: 0.0024 - val_loss: 3.5068e-06 - val_mae: 0.0017 - lr: 0.0010
Epoch 5/30
243/243 [==============================] - 2s 7ms/step - loss: 8.7279e-06 - mae: 0.0021 - val_loss: 1.9400e-05 - val_mae: 0.0040 - lr: 0.0010
Epoch 6/30
243/243 [==============================] - 2s 7ms/step - loss: 7.5734e-06 - mae: 0.0020 - val_loss: 2.1090e-05 - val_mae: 0.0042 - lr: 0.0010
Epoch 7/30
243/243 [==============================] - 2s 7ms/step - loss: 6.8000e

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
243/243 [==============================] - 6s 10ms/step - loss: 2.0906e-04 - mae: 0.0080 - val_loss: 1.6309e-05 - val_mae: 0.0036 - lr: 0.0010
Epoch 2/30
243/243 [==============================] - 2s 7ms/step - loss: 3.5703e-05 - mae: 0.0042 - val_loss: 4.2590e-05 - val_mae: 0.0061 - lr: 0.0010
Epoch 3/30
243/243 [==============================] - 2s 7ms/step - loss: 2.6059e-05 - mae: 0.0036 - val_loss: 7.1098e-05 - val_mae: 0.0082 - lr: 0.0010
Epoch 4/30
243/243 [==============================] - 2s 7ms/step - loss: 1.8830e-05 - mae: 0.0030 - val_loss: 2.4662e-05 - val_mae: 0.0045 - lr: 0.0010
Epoch 5/30
243/243 [==============================] - 2s 8ms/step - loss: 1.4701e-05 - mae: 0.0026 - val_loss: 1.9827e-05 - val_mae: 0.0040 - lr: 0.0010
Epoch 6/30
243/243 [==============================] - 2s 10ms/step - loss: 1.3159e-05 - mae: 0.0025 - val_loss: 8.3410e-06 - val_mae: 0.0024 - lr: 0.0010
Epoch 7/30
243/243 [==============================] - 2s 7ms/step - loss: 1.1824

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
29/29 [==============================] - 4s 37ms/step - loss: 0.0050 - mae: 0.0510 - val_loss: 5.6788e-04 - val_mae: 0.0238 - lr: 0.0010
Epoch 2/30
29/29 [==============================] - 0s 10ms/step - loss: 4.3928e-04 - mae: 0.0165 - val_loss: 2.8692e-05 - val_mae: 0.0053 - lr: 0.0010
Epoch 3/30
29/29 [==============================] - 0s 11ms/step - loss: 2.1515e-04 - mae: 0.0115 - val_loss: 6.2054e-07 - val_mae: 6.1299e-04 - lr: 0.0010
Epoch 4/30
29/29 [==============================] - 0s 13ms/step - loss: 1.8824e-04 - mae: 0.0108 - val_loss: 1.1931e-05 - val_mae: 0.0033 - lr: 0.0010
Epoch 5/30
29/29 [==============================] - 0s 12ms/step - loss: 1.5599e-04 - mae: 0.0097 - val_loss: 3.4765e-06 - val_mae: 0.0017 - lr: 0.0010
Epoch 6/30
29/29 [==============================] - 0s 13ms/step - loss: 1.3950e-04 - mae: 0.0093 - val_loss: 2.6827e-05 - val_mae: 0.0051 - lr: 0.0010
Epoch 7/30
29/29 [==============================] - 0s 12ms/step - loss: 1.2238e-04 - ma

...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
243/243 [==============================] - 7s 16ms/step - loss: 1.0593e-04 - mae: 0.0054 - val_loss: 8.8678e-06 - val_mae: 0.0024 - lr: 0.0010
Epoch 2/30
243/243 [==============================] - 2s 9ms/step - loss: 1.3784e-05 - mae: 0.0026 - val_loss: 1.3276e-05 - val_mae: 0.0028 - lr: 0.0010
Epoch 3/30
243/243 [==============================] - 2s 10ms/step - loss: 8.7306e-06 - mae: 0.0020 - val_loss: 3.4269e-05 - val_mae: 0.0051 - lr: 0.0010
Epoch 4/30
243/243 [==============================] - 2s 9ms/step - loss: 6.5745e-06 - mae: 0.0018 - val_loss: 4.4496e-05 - val_mae: 0.0060 - lr: 0.0010
Epoch 5/30
243/243 [==============================] - 2s 9ms/step - loss: 5.3776e-06 - mae: 0.0016 - val_loss: 1.0830e-05 - val_mae: 0.0026 - lr: 0.0010
Epoch 6/30
243/243 [==============================] - 3s 12ms/step - loss: 5.0790e-06 - mae: 0.0016 - val_loss: 8.7910e-06 - val_mae: 0.0023 - lr: 0.0010
Epoch 7/30
243/243 [==============================] - 3s 11ms/step - loss: 3.68

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
243/243 [==============================] - 7s 17ms/step - loss: 2.0143e-04 - mae: 0.0067 - val_loss: 1.0175e-05 - val_mae: 0.0028 - lr: 0.0010
Epoch 2/30
243/243 [==============================] - 2s 10ms/step - loss: 1.8173e-05 - mae: 0.0031 - val_loss: 4.2363e-06 - val_mae: 0.0017 - lr: 0.0010
Epoch 3/30
243/243 [==============================] - 2s 9ms/step - loss: 1.2714e-05 - mae: 0.0026 - val_loss: 2.9681e-05 - val_mae: 0.0051 - lr: 0.0010
Epoch 4/30
243/243 [==============================] - 2s 9ms/step - loss: 1.0633e-05 - mae: 0.0024 - val_loss: 5.9164e-06 - val_mae: 0.0021 - lr: 0.0010
Epoch 5/30
243/243 [==============================] - 2s 9ms/step - loss: 8.2905e-06 - mae: 0.0021 - val_loss: 2.6500e-06 - val_mae: 0.0014 - lr: 0.0010
Epoch 6/30
243/243 [==============================] - 3s 12ms/step - loss: 7.5292e-06 - mae: 0.0021 - val_loss: 6.3887e-06 - val_mae: 0.0022 - lr: 0.0010
Epoch 7/30
243/243 [==============================] - 3s 11ms/step - loss: 5.97

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## GRU

In [7]:
# Function to build a GRU model with specified input shape
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(128, activation='tanh', input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(64, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [8]:
def evaluate_gru():
    # Define paths for data and model saving
    stock_price = './data/normalized_stock_data/'
    model_save_path = './models/'
    os.makedirs(model_save_path, exist_ok=True)

    # DataFrame to store accuracy metrics
    accuracy_df = pd.DataFrame(columns=['TICKER', 'mse', 'mape'])

    # Load the scaler for the 'Close' column to de-normalize predictions
    scaler_close = joblib.load('./scalers/Close_scaler.pkl')

    # Process each file in the normalized stock data directory
    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)
        df = pd.read_csv(file_path, thousands=',')

        # Define feature and label columns
        feature_cols = ['Close', 'Volume', 'Close_snp500', 'Close_dji', 'Close_nasdaq']
        label_cols = ['Close']

        # Prepare feature and label dataframes and convert to numpy arrays
        feature_df = pd.DataFrame(df, columns=feature_cols)
        label_df = pd.DataFrame(df, columns=label_cols)
        feature_np = feature_df.to_numpy()
        label_np = label_df.to_numpy()

        # Create sequence datasets for the GRU model
        window_size = 30
        X, Y = make_sequence_dataset(feature_np, label_np, window_size)

        # Split data into training and testing sets
        split = -218
        x_train = X[0:split]
        y_train = Y[0:split]
        x_test = X[split:]
        y_test = Y[split:]

        # Build and train the GRU model
        model = build_gru_model(x_train[0].shape)

        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

        model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=64, callbacks=[early_stopping, reduce_lr])

        # Make predictions with the trained model
        pred = model.predict(x_test)

        # De-normalize the actual and predicted values for evaluation
        actual_denormalized = scaler_close.inverse_transform(y_test).flatten()
        pred_denormalized = scaler_close.inverse_transform(pred).flatten()

        # Extract the ticker symbol from the file name for identification
        ticker = os.path.splitext(file)[0]

        # Save the trained model
        model_save_filename = f"{model_save_path}{ticker}_gru_model.h5"
        model.save(model_save_filename)

        # Evaluate the model
        mse = get_mse(actual_denormalized, pred_denormalized)
        mape = get_mape(actual_denormalized, pred_denormalized)
        directional_accuracy = get_directional_accuracy(actual_denormalized, pred_denormalized)

        # Save prediction results to a CSV file
        forecast_file_name = f"./prediction/{ticker}_gru.csv"
        pd.DataFrame({'ds': df['Date'].iloc[-len(pred_denormalized):], 'yhat': pred_denormalized}).to_csv(forecast_file_name, index=False)

        # Update the accuracy DataFrame with the results
        accuracy_df = pd.concat([accuracy_df, pd.DataFrame([[ticker, mse, mape, directional_accuracy]], columns=['TICKER', 'mse', 'mape', 'directional_accuracy'])], ignore_index=True)

    return accuracy_df

In [9]:
accuracy_gru = evaluate_gru()
accuracy_gru.to_csv("gru.csv")

Epoch 1/30
104/104 [==============================] - 7s 13ms/step - loss: 4.8013e-04 - mae: 0.0126 - val_loss: 1.8152e-06 - val_mae: 0.0011 - lr: 0.0010
Epoch 2/30
104/104 [==============================] - 1s 6ms/step - loss: 6.7501e-05 - mae: 0.0054 - val_loss: 4.3182e-06 - val_mae: 0.0018 - lr: 0.0010
Epoch 3/30
104/104 [==============================] - 1s 7ms/step - loss: 3.7545e-05 - mae: 0.0042 - val_loss: 4.1233e-06 - val_mae: 0.0017 - lr: 0.0010
Epoch 4/30
104/104 [==============================] - 1s 6ms/step - loss: 3.0411e-05 - mae: 0.0038 - val_loss: 2.3176e-05 - val_mae: 0.0046 - lr: 0.0010
Epoch 5/30
104/104 [==============================] - 1s 9ms/step - loss: 2.2262e-05 - mae: 0.0033 - val_loss: 5.9889e-06 - val_mae: 0.0021 - lr: 0.0010
Epoch 6/30
7/7 [==============================] - 1s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
122/122 [==============================] - 5s 12ms/step - loss: 3.5432e-04 - mae: 0.0112 - val_loss: 7.8201e-05 - val_mae: 0.0082 - lr: 0.0010
Epoch 2/30
122/122 [==============================] - 1s 7ms/step - loss: 6.9109e-05 - mae: 0.0057 - val_loss: 6.2658e-05 - val_mae: 0.0075 - lr: 0.0010
Epoch 3/30
122/122 [==============================] - 1s 9ms/step - loss: 4.4489e-05 - mae: 0.0047 - val_loss: 7.8712e-06 - val_mae: 0.0024 - lr: 0.0010
Epoch 4/30
122/122 [==============================] - 1s 9ms/step - loss: 2.7789e-05 - mae: 0.0035 - val_loss: 8.7031e-06 - val_mae: 0.0025 - lr: 0.0010
Epoch 5/30
122/122 [==============================] - 1s 9ms/step - loss: 2.1810e-05 - mae: 0.0032 - val_loss: 2.3488e-05 - val_mae: 0.0047 - lr: 0.0010
Epoch 6/30
122/122 [==============================] - 1s 7ms/step - loss: 1.8283e-05 - mae: 0.0029 - val_loss: 1.9799e-05 - val_mae: 0.0043 - lr: 0.0010
Epoch 7/30
122/122 [==============================] - 1s 7ms/step - loss: 1.5843e

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
51/51 [==============================] - 4s 19ms/step - loss: 0.0032 - mae: 0.0385 - val_loss: 0.0026 - val_mae: 0.0503 - lr: 0.0010
Epoch 2/30
51/51 [==============================] - 0s 7ms/step - loss: 7.5750e-04 - mae: 0.0197 - val_loss: 0.0023 - val_mae: 0.0476 - lr: 0.0010
Epoch 3/30
51/51 [==============================] - 0s 7ms/step - loss: 4.1169e-04 - mae: 0.0146 - val_loss: 3.2250e-04 - val_mae: 0.0175 - lr: 0.0010
Epoch 4/30
51/51 [==============================] - 0s 8ms/step - loss: 3.1374e-04 - mae: 0.0127 - val_loss: 4.3455e-04 - val_mae: 0.0205 - lr: 0.0010
Epoch 5/30
51/51 [==============================] - 1s 10ms/step - loss: 2.2433e-04 - mae: 0.0108 - val_loss: 1.4728e-05 - val_mae: 0.0031 - lr: 0.0010
Epoch 6/30
51/51 [==============================] - 0s 9ms/step - loss: 1.9375e-04 - mae: 0.0101 - val_loss: 7.0083e-05 - val_mae: 0.0076 - lr: 0.0010
Epoch 7/30
51/51 [==============================] - 0s 9ms/step - loss: 1.6725e-04 - mae: 0.0092 - val_l

...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
122/122 [==============================] - 5s 12ms/step - loss: 4.9032e-04 - mae: 0.0120 - val_loss: 3.0176e-05 - val_mae: 0.0051 - lr: 0.0010
Epoch 2/30
122/122 [==============================] - 1s 7ms/step - loss: 7.4633e-05 - mae: 0.0057 - val_loss: 4.5788e-05 - val_mae: 0.0065 - lr: 0.0010
Epoch 3/30
122/122 [==============================] - 1s 7ms/step - loss: 4.2314e-05 - mae: 0.0044 - val_loss: 9.3014e-06 - val_mae: 0.0024 - lr: 0.0010
Epoch 4/30
122/122 [==============================] - 1s 7ms/step - loss: 2.9581e-05 - mae: 0.0037 - val_loss: 1.3710e-05 - val_mae: 0.0031 - lr: 0.0010
Epoch 5/30
122/122 [==============================] - 1s 7ms/step - loss: 2.2231e-05 - mae: 0.0032 - val_loss: 4.5525e-06 - val_mae: 0.0018 - lr: 0.0010
Epoch 6/30
122/122 [==============================] - 1s 9ms/step - loss: 1.7646e-05 - mae: 0.0029 - val_loss: 4.4218e-06 - val_mae: 0.0018 - lr: 0.0010
Epoch 7/30
122/122 [==============================] - 1s 10ms/step - loss: 1.5361

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/30
96/96 [==============================] - 4s 14ms/step - loss: 0.0015 - mae: 0.0221 - val_loss: 8.9383e-05 - val_mae: 0.0094 - lr: 0.0010
Epoch 2/30
96/96 [==============================] - 1s 7ms/step - loss: 1.8376e-04 - mae: 0.0092 - val_loss: 3.4818e-05 - val_mae: 0.0058 - lr: 0.0010
Epoch 3/30
96/96 [==============================] - 1s 7ms/step - loss: 1.0032e-04 - mae: 0.0068 - val_loss: 2.0303e-06 - val_mae: 0.0012 - lr: 0.0010
Epoch 4/30
96/96 [==============================] - 1s 7ms/step - loss: 7.3933e-05 - mae: 0.0059 - val_loss: 5.2604e-05 - val_mae: 0.0072 - lr: 0.0010
Epoch 5/30
96/96 [==============================] - 1s 7ms/step - loss: 5.4428e-05 - mae: 0.0050 - val_loss: 1.6739e-05 - val_mae: 0.0040 - lr: 0.0010
Epoch 6/30
96/96 [==============================] - 1s 7ms/step - loss: 3.8456e-05 - mae: 0.0042 - val_loss: 7.1097e-06 - val_mae: 0.0026 - lr: 0.0010
Epoch 7/30
96/96 [==============================] - 1s 8ms/step - loss: 3.4522e-05 - mae: 0.0040 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Combine model

## Make dataframe for each ticker's with models' mse and mape

In [10]:
# Load results from CSV files for each forecasting method
arima_df = pd.read_csv('arima.csv')
prophet_df = pd.read_csv('prophet.csv')
lstm_df = pd.read_csv('lstm.csv')
gru_df = pd.read_csv('gru.csv')

# Assign a 'Method' column to each DataFrame to identify the forecasting method
arima_df['Method'] = 'ARIMA'
lstm_df['Method'] = 'LSTM'
prophet_df['Method'] = 'Prophet'
gru_df['Method'] = 'GRU'

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat([arima_df, lstm_df, prophet_df, gru_df])

# Sort combined_df by MAPE and group by 'TICKER' to find the best method for each ticker
best_methods = combined_df.sort_values('mape').groupby('TICKER').first().reset_index()
# Drop any unnamed column that might be present due to the CSV structure
best_methods.drop('Unnamed: 0', axis=1, inplace=True)

print(best_methods)

    TICKER         mse      mape  directional_accuracy Method
0     AAPL   71.888697  4.078712              0.497696    GRU
1     ABBV  112.458241  5.685523              0.479263    GRU
2     ABNB  133.955107  7.609834              0.576037    GRU
3      ABT   45.913953  5.383169              0.493088   LSTM
4      ACN   57.940706  2.152783              0.525346    GRU
..     ...         ...       ...                   ...    ...
103   WELL    2.916102  1.726043              0.525346    GRU
104    WFC    2.828355  3.240042              0.520737    GRU
105    WMT    3.867226  1.047887              0.502304    GRU
106    XEL    2.744028  2.080550              0.502304   LSTM
107    XOM    7.703764  2.052047              0.451613    GRU

[108 rows x 5 columns]


In [11]:
# Count how many times each method was best across all tickers
method_counts = best_methods['Method'].value_counts()
print(method_counts)

GRU        61
LSTM       42
Prophet     3
ARIMA       2
Name: Method, dtype: int64


In [12]:
# Calculate and print the total average MAPE across all methods and tickers
average_mape = combined_df['mape'].mean()
print("Total average MAPE:", average_mape)

Total average MAPE: 10.32377699148769


In [13]:
# Drop any unnamed column and rename columns to clearly indicate the method and metric
arima_df = pd.read_csv('arima.csv')
prophet_df = pd.read_csv('prophet.csv')
lstm_df = pd.read_csv('lstm.csv')
gru_df = pd.read_csv('gru.csv')

# Repeat the loading step to ensure clean data, then adjust and rename
arima_df = arima_df.drop('Unnamed: 0', axis=1)
arima_df.columns = ['TICKER', 'mse_arima', 'mape_arima', 'dir_arima']

gru_df = gru_df.drop('Unnamed: 0', axis=1)
gru_df.columns = ['TICKER', 'mse_gru', 'mape_gru', 'dir_gru']

lstm_df = lstm_df.drop('Unnamed: 0', axis=1)
lstm_df.columns = ['TICKER', 'mse_lstm', 'mape_lstm', 'dir_lstm']

prophet_df = prophet_df.drop('Unnamed: 0', axis=1)
prophet_df.columns = ['TICKER', 'mse_prophet', 'mape_prophet', 'dir_prophet']

# Merge all DataFrames on 'TICKER' to create a comprehensive DataFrame for all methods
merged_df = arima_df.merge(gru_df, on='TICKER').merge(lstm_df, on='TICKER').merge(prophet_df, on='TICKER')

merged_df

,TICKER,mse_arima,mape_arima,dir_arima,mse_gru,mape_gru,dir_gru,mse_lstm,mape_lstm,dir_lstm,mse_prophet,mape_prophet,dir_prophet
0,APD,519.285125,7.227040,0.027778,48.266708,1.824004,0.502304,187.618884,3.523659,0.423963,198.061393,3.976113,0.532407
1,CRH,171.300307,22.934341,0.023148,19.090478,7.295543,0.488479,13.270352,6.019763,0.511521,65.854811,12.769471,0.407407
2,ECL,700.928252,13.678644,0.027778,17.399464,2.029997,0.437788,14.496949,1.793848,0.456221,2276.791119,28.033827,0.462963
3,DOW,19.733414,6.544109,0.013889,23.344400,7.902990,0.488479,6.526309,3.639030,0.474654,109.307711,18.532557,0.527778
4,FCX,11.761375,7.082749,0.004630,35.627120,12.582039,0.410138,5.152092,4.853978,0.516129,18.490965,8.533245,0.509259
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,NEE,218.807799,19.243211,0.013889,75.593713,10.734524,0.433180,13.493105,4.161401,0.451613,234.565277,19.543124,0.481481
104,PCG,0.596448,3.926725,0.023148,2.828162,8.301625,0.543779,1.404805,6.456897,0.483871,33.857689,34.328294,0.430556
105,SO,12.975567,4.340712,0.018519,2.691717,1.965448,0.506912,6.247638,3.075098,0.460829,8.940615,3.497039,0.555556
106,SRE,23.203298,5.632522,0.018519,3.324990,1.967446,0.516129,3.303124,2.065608,0.428571,30.941998,6.596207,0.453704


## Calculate the weights for each models for ticker based on mse and mape

In [14]:
# Copy relevant columns for weight calculation
weights = merged_df[['mse_arima', 'mape_arima', 'dir_arima', 'mse_gru', 'mape_gru', 'dir_gru', 'mse_lstm', 'mape_lstm', 'dir_lstm', 'mse_prophet', 'mape_prophet', 'dir_prophet']].copy()

# Invert MSE and MAPE values for weighting (since lower values are better)
weights[['mse_arima', 'mape_arima', 'mse_gru', 'mape_gru',  'mse_lstm', 'mape_lstm', 'mse_prophet', 'mape_prophet']]= weights[['mse_arima', 'mape_arima', 'mse_gru', 'mape_gru',  'mse_lstm', 'mape_lstm', 'mse_prophet', 'mape_prophet']].apply(lambda x: 1 / x)

# Apply a scaling factor to directional accuracy to adjust its impact
dir_acc_scale_factor = 0.2

# Scale the directional accuracy by the factor
weights['dir_arima_scaled'] = weights['dir_arima'] * dir_acc_scale_factor
weights['dir_gru_scaled'] = weights['dir_gru'] * dir_acc_scale_factor
weights['dir_lstm_scaled'] = weights['dir_lstm'] * dir_acc_scale_factor
weights['dir_prophet_scaled'] = weights['dir_prophet'] * dir_acc_scale_factor

# Calculate combined weights for each model, then normalize these weights
weights['weight_arima'] = weights[['mse_arima', 'mape_arima', 'dir_arima_scaled']].mean(axis=1)
weights['weight_gru'] = weights[['mse_gru', 'mape_gru', 'dir_gru_scaled']].mean(axis=1)
weights['weight_lstm'] = weights[['mse_lstm', 'mape_lstm', 'dir_lstm_scaled']].mean(axis=1)
weights['weight_prophet'] = weights[['mse_prophet', 'mape_prophet', 'dir_prophet_scaled']].mean(axis=1)

# Normalize the weights
weights_normalized = weights[['weight_arima', 'weight_gru', 'weight_lstm', 'weight_prophet']].div(weights[['weight_arima', 'weight_gru', 'weight_lstm', 'weight_prophet']].sum(axis=1), axis=0)

# Add 'TICKER' column to normalized weights and reorder columns for clarity
weights_normalized['TICKER'] = merged_df['TICKER']
weights_normalized = weights_normalized[['TICKER', 'weight_arima', 'weight_gru', 'weight_lstm', 'weight_prophet']]

# Save the weights to a CSV for use in ensemble modeling
weights_normalized.to_csv("model_weight.csv", index=False)

# Display the final DataFrame of normalized weights
weights_normalized

,TICKER,weight_arima,weight_gru,weight_lstm,weight_prophet
0,APD,0.093962,0.431267,0.240892,0.233879
1,CRH,0.062874,0.333902,0.399755,0.203469
2,ECL,0.051204,0.407669,0.458843,0.082285
3,DOW,0.177057,0.229252,0.448909,0.144782
4,FCX,0.190365,0.158883,0.421849,0.228903
...,...,...,...,...,...
103,NEE,0.073336,0.238651,0.500416,0.187596
104,PCG,0.533750,0.160686,0.265647,0.039917
105,SO,0.130780,0.412616,0.242698,0.213905
106,SRE,0.098230,0.399439,0.382067,0.120263


# Forecast

## Prophet

In [15]:
from prophet import Prophet

In [16]:
def forecast_prophet():
    stock_price = './data/stock_price_data/'
    forecast_folder = './forecast/'

    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)

        # Extract ticker from the file name
        ticker = os.path.splitext(file)[0]

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Extract 'Close' column and reset the index
        df_prophet = df['Close'].reset_index()

        # Rename the columns to match Prophet's requirements
        df_prophet.columns = ['ds', 'y']

        # Convert the 'ds' column to datetime
        df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

        # Create and fit the Prophet model using the entire dataset
        model = Prophet()
        model.fit(df_prophet)

        # Make future DataFrame for forecasting 56 days into the future
        future = model.make_future_dataframe(periods=56)

        # Forecast using the fitted model
        forecast = model.predict(future)

        # Extract only the future forecast (last 56 days)
        forecast_future = forecast.iloc[-56:][['ds', 'yhat']]

        # Save the forecast data to a CSV file
        forecast_future.to_csv(f"{forecast_folder}{ticker}_prophet.csv", index=False)

In [17]:
forecast_prophet()

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt8j508zc/_2meugvs.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt8j508zc/uxu21ul8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=91860', 'data', 'file=/tmp/tmpt8j508zc/_2meugvs.json', 'init=/tmp/tmpt8j508zc/uxu21ul8.json', 'output', 'file=/tmp/tmpt8j508zc/prophet_modelh_bdoc1t/prophet_model-20240220002356.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:23:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:24:01 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt8

## ARIMA

In [18]:
from statsmodels.tsa.arima.model import ARIMA

In [21]:
def forecast_arima():
    stock_price = './data/stock_price_data/'
    forecast_folder = './forecast/'

    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)

        # Extract ticker from the file name
        ticker = os.path.splitext(file)[0]

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

        # Extract 'Close' column and reset the index
        df_arima = df['Close'].reset_index()

        # Rename the columns to match Prophet's requirements
        df_arima.columns = ['ds', 'y']

        # Convert the 'ds' column to datetime
        df_arima['ds'] = pd.to_datetime(df_arima['ds'])

        # Use the entire dataset for training
        train = df_arima

        order = (1,1,0)

        # Fit ARIMA model using the training data
        model_arima = ARIMA(train['y'], order=order)
        results_arima = model_arima.fit()

        # Forecast for the next 56 days
        forecast = results_arima.get_forecast(steps=56)
        y_pred = forecast.predicted_mean.values

        # Creating a forecast DataFrame with future dates
        last_date = train['ds'].iloc[-1]
        future_dates = pd.date_range(start=last_date, periods=57, closed='right')
        forecast_df = pd.DataFrame({'ds': future_dates, 'yhat': y_pred})

        # Save the forecast to a CSV file
        forecast_df.to_csv(f"{forecast_folder}{ticker}_arima.csv", index=False)

In [22]:
forecast_arima()

<ipython-input-21-663a239243bc>:38: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  future_dates = pd.date_range(start=last_date, periods=57, closed='right')
<ipython-input-21-663a239243bc>:38: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  future_dates = pd.date_range(start=last_date, periods=57, closed='right')
<ipython-input-21-663a239243bc>:38: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  future_dates = pd.date_range(start=last_date, periods=57, closed='right')
<ipython-input-21-663a239243bc>:38: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  future_dates = pd.date_range(start=last_date, periods=57, closed='right')
<ipython-input-21-663a239243bc>:38: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  future_dates = pd.date_range(start=last_date, periods=57, closed='right')
<ipython-input-21-663a239243bc>:38: FutureWarning: Argument `closed` is deprecat

## LSTM

In [23]:
from keras.models import load_model

In [24]:
def forecast_lstm():
    # Define the directories for normalized stock data, saved models, and forecast output
    stock_price = './data/normalized_stock_data/'
    model_save_path = './models/'
    forecast_folder = './forecast/'

    # Load the scaler used for the 'Close' column to denormalize the predictions later
    scaler_close = joblib.load('./scalers/Close_scaler.pkl')

    # Iterate over each file in the normalized stock data directory
    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)
        df = pd.read_csv(file_path, thousands=',')

        # Specify the columns to be used as features and labels
        feature_cols = ['Close', 'Volume', 'Close_snp500', 'Close_dji', 'Close_nasdaq']
        label_cols = ['Close']

        # Convert the specified columns into NumPy arrays for processing
        feature_df = pd.DataFrame(df, columns=feature_cols)
        label_df = pd.DataFrame(df, columns=label_cols)

        feature_np = feature_df.to_numpy()
        label_np = label_df.to_numpy()

        # Extract the ticker symbol from the file name and load the corresponding LSTM model
        ticker = os.path.splitext(file)[0]
        model = load_model(f"{model_save_path}{ticker}_lstm_model.h5")

        # Define the window size used during training for sequence prediction
        window_size = 40

        # Prepare the initial sequence batch for prediction
        current_batch = feature_np[-window_size:].reshape(1, window_size, len(feature_cols))

        # Initialize a list to store predictions
        predictions = []
        # Predict 56 future points
        for _ in range(56):
            # Predict the next value using the current batch
            current_pred = model.predict(current_batch)[0]

            # Update the current batch to include the new prediction
            new_features = np.copy(current_batch[:, -1, :])  # Copy the last features
            new_features[0, 0] = current_pred  # Update 'Close' value in the features
            new_features_shaped = np.reshape(new_features, (1, 1, len(feature_cols)))
            current_batch = np.append(current_batch[:, 1:, :], new_features_shaped, axis=1)

            # Add the prediction to the list
            predictions.append(current_pred)

        # Denormalize the predictions to convert them back to their original scale
        predictions_denormalized = scaler_close.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

        # Generate future dates starting from the last known date in the dataset
        last_date = pd.to_datetime(df['Date'].iloc[-1])
        future_dates = [last_date + pd.Timedelta(days=i) for i in range(1, 57)]

        # Save the forecasted data to a CSV file
        forecast_df = pd.DataFrame({'ds': future_dates, 'yhat': predictions_denormalized})
        forecast_file_name = f"{forecast_folder}{ticker}_lstm.csv"
        forecast_df.to_csv(forecast_file_name, index=False)

In [25]:
forecast_lstm()

1/1 [==============================] - 0s 19ms/step


<ipython-input-24-749358b74a45>:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_features[0, 0] = current_pred  # Update 'Close' value in the features


1/1 [==============================] - 0s 24ms/step


<ipython-input-24-749358b74a45>:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_features[0, 0] = current_pred  # Update 'Close' value in the features


1/1 [==============================] - 0s 18ms/step


...

1/1 [==============================] - 0s 22ms/step


<ipython-input-24-749358b74a45>:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_features[0, 0] = current_pred  # Update 'Close' value in the features


1/1 [==============================] - ETA: 0s

<ipython-input-24-749358b74a45>:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_features[0, 0] = current_pred  # Update 'Close' value in the features


1/1 [==============================] - 0s 20ms/step


## GRU

In [26]:
def forecast_gru():
    # Define directories for the normalized stock data, saved models, and where forecasts will be saved
    stock_price = './data/normalized_stock_data/'
    model_save_path = './models/'
    forecast_folder = './forecast/'

    # Load the scaler for the 'Close' column used during data normalization
    scaler_close = joblib.load('./scalers/Close_scaler.pkl')

    # Iterate through each file in the normalized stock data directory
    for file in os.listdir(stock_price):
        file_path = os.path.join(stock_price, file)
        df = pd.read_csv(file_path, thousands=',')

        # Select the features and labels for the prediction
        feature_cols = ['Close', 'Volume', 'Close_snp500', 'Close_dji', 'Close_nasdaq']
        label_cols = ['Close']

        # Convert these columns into NumPy arrays for processing
        feature_df = pd.DataFrame(df, columns=feature_cols)
        label_df = pd.DataFrame(df, columns=label_cols)
        feature_np = feature_df.to_numpy()
        label_np = label_df.to_numpy()

        # Load the pre-trained GRU model for the specific stock
        ticker = os.path.splitext(file)[0]
        model = load_model(f"{model_save_path}{ticker}_gru_model.h5")

        # Define the window size used for the input sequence
        window_size = 30

        # Prepare the initial sequence batch for making predictions
        current_batch = feature_np[-window_size:].reshape(1, window_size, len(feature_cols))

        predictions = []
        # Predict 56 data points
        for _ in range(56):
            # Make a prediction using the current batch of features
            current_pred = model.predict(current_batch)[0]

            # Update the batch to include the new prediction
            new_features = np.copy(current_batch[:, -1, :])
            new_features[0, 0] = current_pred # Update 'Close' feature with the new prediction
            new_features_shaped = np.reshape(new_features, (1, 1, len(feature_cols)))
            current_batch = np.append(current_batch[:, 1:, :], new_features_shaped, axis=1)

            predictions.append(current_pred)

        # Denormalize the predictions to convert them back to the original scale
        predictions_denormalized = scaler_close.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

        # Generate future dates starting from the last known date in the dataset
        last_date = pd.to_datetime(df['Date'].iloc[-1])
        future_dates = [last_date + pd.Timedelta(days=i) for i in range(1, 57)]

        # Save the forecast data to a CSV file for further analysis
        forecast_df = pd.DataFrame({'ds': future_dates, 'yhat': predictions_denormalized})
        forecast_file_name = f"{forecast_folder}{ticker}_gru.csv"
        forecast_df.to_csv(forecast_file_name, index=False)

In [27]:
forecast_gru()

1/1 [==============================] - 0s 29ms/step


<ipython-input-26-19466d823fb2>:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_features[0, 0] = current_pred # Update 'Close' feature with the new prediction


1/1 [==============================] - 0s 18ms/step


<ipython-input-26-19466d823fb2>:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_features[0, 0] = current_pred # Update 'Close' feature with the new prediction


1/1 [==============================] - 0s 27ms/step


...

1/1 [==============================] - 0s 18ms/step


<ipython-input-26-19466d823fb2>:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_features[0, 0] = current_pred # Update 'Close' feature with the new prediction


1/1 [==============================] - 0s 20ms/step


<ipython-input-26-19466d823fb2>:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_features[0, 0] = current_pred # Update 'Close' feature with the new prediction


1/1 [==============================] - 0s 21ms/step


# Create an ensemble model based on models' weight

## Create a dataframe consist of Date, Close, 4 models' prediction, ensemble model's prediction for validation and test set

In [28]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Load the weights data
weights_df = pd.read_csv('model_weight.csv')

# Directory paths
stock_dataset = './data/stock_price_data'
prediction_dir = './prediction'
ensemble_dir = './ensemble'

# Ensure ensemble directory exists
if not os.path.exists(ensemble_dir):
    os.makedirs(ensemble_dir)

# List of models
models = ['prophet', 'arima', 'lstm', 'gru']

def process_ticker(ticker, weights_df):
    # Load actual stock data
    stock_file = f'{stock_dataset}/{ticker}.csv'
    stock_df = pd.read_csv(stock_file)

    # Initialize DataFrame for merged data
    merged_df = stock_df[['Date', 'Close']].tail(217)
    merged_df['Date'] = pd.to_datetime(merged_df['Date'])

    # Load predictions and merge
    for model in models:
        model_file = f'{prediction_dir}/{ticker}_{model}.csv'
        model_df = pd.read_csv(model_file)
        merged_df[model.capitalize()] = model_df['yhat'].head(217).values

    # Calculate ensemble predictions
    ticker_weights = weights_df[weights_df['TICKER'] == ticker].iloc[0]
    ensemble_predictions = sum(ticker_weights[f'weight_{model}'] * merged_df[model.capitalize()] for model in models)

    # Add ensemble predictions to DataFrame
    merged_df['Ensemble'] = ensemble_predictions

    # Save merged data to CSV
    merged_file_path = os.path.join(ensemble_dir, f'{ticker}_merged.csv')
    merged_df.to_csv(merged_file_path, index=False)

# Process each ticker
for ticker in weights_df['TICKER']:
    process_ticker(ticker, weights_df)


In [29]:
import pandas as pd
import os

# Load weight data
weights_df = pd.read_csv('model_weight.csv')

# Directory paths
stock_dataset = './data/stock_price_data'
prediction_dir = './prediction'
forecast_dir = './forecast'
test_data_dir = './data/test_data'
ensemble_dir = './ensemble'

# Ensure ensemble directory exists
if not os.path.exists(ensemble_dir):
    os.makedirs(ensemble_dir)

# Model names
models = ['arima', 'gru', 'lstm', 'prophet']

def update_merged_data(ticker):
    # Load the existing merged file
    merged_file_path = os.path.join(ensemble_dir, f'{ticker}_merged.csv')
    merged_df = pd.read_csv(merged_file_path)

    # Load test data
    test_data_file = os.path.join(test_data_dir, f'{ticker}.csv')
    test_data_df = pd.read_csv(test_data_file)

    # Prepare a DataFrame for new data including Date, Close, and model predictions
    new_data = test_data_df[['Date', 'Close']].copy()

    # Load forecast data for each model and append the top 56 predictions
    for model in models:
        forecast_path = os.path.join(forecast_dir, f'{ticker}_{model}.csv')
        forecast_df = pd.read_csv(forecast_path).head(56)  # Take top 56 predictions
        new_data[model.capitalize()] = forecast_df['yhat'].values  # Add model predictions as new columns

    # Calculate the ensemble predictions using the weights
    weights = weights_df.loc[weights_df['TICKER'] == ticker, [f'weight_{model}' for model in models]].iloc[0].values
    new_data['Ensemble'] = sum(new_data[model.capitalize()] * weight for model, weight in zip(models, weights))

    # Append new data to the existing merged DataFrame
    updated_merged_df = pd.concat([merged_df, new_data], ignore_index=True)

    # Save the updated merged data
    updated_merged_df.to_csv(merged_file_path, index=False)
    print(f"Updated merged data with forecasts saved for {ticker}")

# Update merged data for each ticker
for ticker in weights_df['TICKER']:
    update_merged_data(ticker)


Updated merged data with forecasts saved for APD
Updated merged data with forecasts saved for CRH
Updated merged data with forecasts saved for ECL
Updated merged data with forecasts saved for DOW
Updated merged data with forecasts saved for FCX
Updated merged data with forecasts saved for NUE
Updated merged data with forecasts saved for LIN
Updated merged data with forecasts saved for SCCO
Updated merged data with forecasts saved for SHW
Updated merged data with forecasts saved for CTVA
Updated merged data with forecasts saved for CMCSA
Updated merged data with forecasts saved for CHTR
Updated merged data with forecasts saved for DIS
Updated merged data with forecasts saved for META
Updated merged data with forecasts saved for GOOG
Updated merged data with forecasts saved for NFLX
Updated merged data with forecasts saved for T
Updated merged data with forecasts saved for TMUS
Updated merged data with forecasts saved for VZ
Updated merged data with forecasts saved for ABNB
Updated merge